# TASK: Matching Predictions to Sensor Readings - DABBEL

We have 2 tables with sensor data from different time stamps. One table contains actual temperature values recorded, the other contains model predictions. 

The task is to match the readings to the predictions with the closest timestamp and vice-versa, with no repetitions from either side.

I have used pandas dataframes to solve this problem as the pandas library provides essential functions that can be used to perform this task.

In [5]:
import pandas as pd
import numpy as np

#Function to match prdictions and readings tables
def match_predictions_readings(readings, predictions):
  
  #Resetting index of readings df and sorting by the date values
  readings=pd.melt(readings.reset_index(), var_name="Sensor", id_vars='index', value_name="Temperature")
  readings['index_r']=pd.to_datetime(readings['index'])
  readings.drop(columns=['index'], inplace=True)
  readings=readings.sort_values(by='index_r')
  
  #Resetting index of predictions df, Dropping NAs and sorting by the date values
  predictions= pd.melt(predictions.reset_index(), var_name="Sensor", id_vars='index', value_name="Prediction")
  predictions.dropna(inplace=True)
  predictions['index_p']= pd.to_datetime(predictions['index'])
  predictions.drop(columns=['index'], inplace=True)
  predictions=predictions.sort_values(by='index_p')
  
  #Merging the 2 dfs by ensuring same sensor name and nearest time values.
  merged=pd.merge_asof(readings,predictions, by='Sensor', left_on='index_r', right_on='index_p', direction='nearest')
  
  #Calculating time difference between the two indices and sorting in ascending order
  merged['Difference']=np.abs(merged['index_p']-merged['index_r'])
  merged=merged.sort_values(by='Difference')
  
  #Dropping duplicates from columns that came from predictions (to ensure one-to-one mapping) 
  #and keeping first occurences(those values with the smallest time difference)
  merged.drop_duplicates(subset=['index_p','Sensor','Prediction'], keep='first', inplace=True)
  merged=merged.set_index('index_r').drop(columns=['index_p','Difference'])
  merged.index.name=None

  return merged

In [6]:
#TEST 1 - Provided dataframe from task description

readings=pd.DataFrame(index=["2020-12-01 00:00:00","2020-12-01 00:11:34", "2020-12-01 00:20:00"],
                        data={"Sensor1": [20.0,20.1,20.0],
                        "Sensor2": [19.4,19.4,19.4],
                        "Sensor3":[21.3,21.2,21.0],
                        "Sensor4":[20.5,20.4,20.6]})

predictions=pd.DataFrame(index=["2020-12-01 00:00:30","2020-12-01 00:07:24","2020-12-01 00:17:15",
                                "2020-12-01 00:19:00","2020-12-01 00:21:30"],
                        data={"Sensor1": [np.nan,20.2,19.9,20.0,20.1],
                        "Sensor2": [19.5,np.nan,19.6,np.nan,np.nan],
                        "Sensor3":[21.1,np.nan,np.nan,np.nan,np.nan],
                        "Sensor4":[20.3,20.2,20.4,np.nan,20.5]})

print("READINGS\n",readings)
print("\nPREDICTIONS\n",predictions)

match_predictions_readings(readings, predictions)

READINGS
                      Sensor1  Sensor2  Sensor3  Sensor4
2020-12-01 00:00:00     20.0     19.4     21.3     20.5
2020-12-01 00:11:34     20.1     19.4     21.2     20.4
2020-12-01 00:20:00     20.0     19.4     21.0     20.6

PREDICTIONS
                      Sensor1  Sensor2  Sensor3  Sensor4
2020-12-01 00:00:30      NaN     19.5     21.1     20.3
2020-12-01 00:07:24     20.2      NaN      NaN     20.2
2020-12-01 00:17:15     19.9     19.6      NaN     20.4
2020-12-01 00:19:00     20.0      NaN      NaN      NaN
2020-12-01 00:21:30     20.1      NaN      NaN     20.5


,Sensor,Temperature,Prediction
2020-12-01 00:00:00,Sensor2,19.4,19.5
2020-12-01 00:00:00,Sensor3,21.3,21.1
2020-12-01 00:00:00,Sensor4,20.5,20.3
2020-12-01 00:20:00,Sensor1,20.0,20.0
2020-12-01 00:20:00,Sensor4,20.6,20.5
2020-12-01 00:20:00,Sensor2,19.4,19.6
2020-12-01 00:11:34,Sensor1,20.1,20.2
2020-12-01 00:11:34,Sensor4,20.4,20.2


In [30]:
#TEST 2: Predictions size(15000,500) Readings size(3000,500)

#function to generate a list of random timestamps in a given range
def random_datetimes(start, end, n):
  frmt='%d-%m-%Y %H:%M:%S'
  stime=datetime.datetime.strptime(start, frmt)
  etime=datetime.datetime.strptime(end, frmt)
  td=etime-stime
  return [(random.random()*td+stime).replace(microsecond=0) for _ in range(n)]

#Creating list of sensor column names
sensor_list= ['Sensor'+str(i) for i in range(1,501)]

#Creating readings and temperature dataframes with random values. 
#Assuming temperature range (-20,45)
#Assuming date range for the year 2020
readings=pd.DataFrame(data=np.round(np.random.uniform(-20,45,size=(3000, 500)), decimals=1),
                index= random_datetimes(start='01-01-2020 00:00:00', end='31-12-2020 23:59:59', n=3000),
                columns=sensor_list)
predictions=pd.DataFrame(data=np.round(np.random.uniform(-20,45,size=(15000, 500)), decimals=1),
                index= random_datetimes(start='01-01-2020 00:00:00', end='31-12-2020 23:59:59', n=15000),   
                columns=sensor_list)

match_predictions_readings(readings, predictions)

,Sensor,Temperature,Prediction
2020-05-20 16:27:09,Sensor472,-11.4,-0.8
2020-05-20 16:27:09,Sensor294,-17.2,4.3
2020-05-20 16:27:09,Sensor275,21.1,37.3
2020-05-20 16:27:09,Sensor348,31.5,26.1
2020-05-20 16:27:09,Sensor344,38.7,22.0
...,...,...,...
2020-08-19 03:09:55,Sensor400,39.4,-11.1
2020-08-19 03:09:55,Sensor297,21.8,27.2
2020-08-19 03:09:55,Sensor119,0.1,29.5
2020-08-19 03:09:55,Sensor429,-0.2,13.8


Time complexity of the algorithm - This code doesn't use any explicit nested for-loops. Hence, the complexity is O(n) for the code lines that do not use any pandas functions.


The complexity of the entire program should be equivalent to the most time consuming pandas operation. This could mean the time complexity of sort, merge, or melt functions from the pandas library.